In [4]:
import sqlite3
import pandas as pd
con = sqlite3.connect('fire.db')



In [6]:
pd.read_sql('select * from [ETH-USD]', con, index_col = 'trade_id')


,price,last_size,sequence,side,time
trade_id,,,,,
22683468.0,803.92000000,2.67932913,1748869724,sell,2017-12-21T02:22:52.315000Z
22683469.0,803.92000000,6.62057087,1748869726,sell,2017-12-21T02:22:52.315000Z
22683471.0,803.93000000,0.10433540,1748869838,buy,2017-12-21T02:22:55.467000Z
22683472.0,803.92000000,1.67620000,1748870011,sell,2017-12-21T02:23:05.136000Z
22683474.0,803.93000000,2.37220720,1748870039,buy,2017-12-21T02:23:07.446000Z
22683475.0,803.93000000,1.23973482,1748870062,buy,2017-12-21T02:23:10.498000Z
22683476.0,803.93000000,0.00186025,1748870164,buy,2017-12-21T02:23:17.654000Z
22683477.0,803.93000000,0.06200844,1748870169,buy,2017-12-21T02:23:17.738000Z
22683478.0,803.93000000,0.24803377,1748870251,buy,2017-12-21T02:23:21.943000Z


In [48]:
import sql_tools 
import pypyodbc
import pandas as pd
import numpy as np
import ryan_tools as rt
import seaborn as sea
import sqlite3
from datetime import datetime, timedelta
from IPython.display import display,Markdown, HTML
from matplotlib import pyplot as plt
phi = (1 + np.sqrt(5))/2

fire = sql_tools.IcePick(sqlite3, 'fire.db')

def printmd(text):
    if type(text) == pd.DataFrame:
        display(HTML(text.to_html()))
        return
    display(Markdown(text))

In [50]:
data =fire.get_data("ETH-USD")
data.set_index('trade_id', inplace = True)
data['time'] = pd.to_datetime(data['time'])

In [54]:
def to_candles(data):
    
    data['minutes'] =  (pd.to_datetime(data['time'].dt.strftime('%Y-%m-%d %H:%M')))
    candles = pd.DataFrame(index = set(data['minutes']))
    grouped = data.groupby('minutes')
    candles['low'] = grouped['price'].min()
    candles['high'] = grouped['price'].max()
    candles['sell_volume'] = data.groupby(['minutes', 'side'])['last_size'].sum()['sell']
    return candles

x = to_candles(data.copy())

KeyError: 'sell'

In [73]:
data

,price,last_size,sequence,side,time,minutes
trade_id,,,,,,
22683468.0,803.92000000,2.67932913,1748869724,sell,2017-12-21 02:22:52.315,2017-12-21 02:22:00
22683469.0,803.92000000,6.62057087,1748869726,sell,2017-12-21 02:22:52.315,2017-12-21 02:22:00
22683471.0,803.93000000,0.10433540,1748869838,buy,2017-12-21 02:22:55.467,2017-12-21 02:22:00
22683472.0,803.92000000,1.67620000,1748870011,sell,2017-12-21 02:23:05.136,2017-12-21 02:23:00
22683474.0,803.93000000,2.37220720,1748870039,buy,2017-12-21 02:23:07.446,2017-12-21 02:23:00
22683475.0,803.93000000,1.23973482,1748870062,buy,2017-12-21 02:23:10.498,2017-12-21 02:23:00
22683476.0,803.93000000,0.00186025,1748870164,buy,2017-12-21 02:23:17.654,2017-12-21 02:23:00
22683477.0,803.93000000,0.06200844,1748870169,buy,2017-12-21 02:23:17.738,2017-12-21 02:23:00
22683478.0,803.93000000,0.24803377,1748870251,buy,2017-12-21 02:23:21.943,2017-12-21 02:23:00


In [72]:
data['last_size'].sum()

'2.679329136.620570870.104335401.676200002.372207201.239734820.001860250.062008440.248033770.774559875.301595651.0000000019.000000003.886604350.010876281.000000000.635829130.005000000.099852020.442318851.038022141.545114010.996024821.249000000.226000000.011222750.207074692.741480130.0018602510.714335980.600000001.804174112.093784974.6856532610.000000000.500000000.050000003.928713491.240646360.010000005.119037802.0000000010.291690450.0018602516.496000000.005208700.124016880.206560430.001922260.186582320.844883910.571173750.138775497.0007533650.000000008.723651010.070000002.000000001.756848991.000000000.001860250.003512363.100422211.240168880.011161519.046945161.724154847.813355270.012401680.031004220.001860250.028074510.001865850.001865850.001865850.001865850.001865850.005597570.001865850.001865850.001865850.001865850.001865850.001865850.031004228.171500288.437699720.183850008.202500000.013101440.001865850.001865850.001865850.005597570.001865850.039304320.001860250.001865850.001865850.0

In [70]:
data.groupby(['minutes', 'side'])['last_size'].sum().

minutes
2017-12-21 02:22:00                                 2.679329136.62057087
2017-12-21 02:23:00    1.676200001.000000000.635829130.005000000.0998...
2017-12-21 02:24:00    0.050000003.928713491.240646362.000000000.2065...
2017-12-21 02:25:00    7.813355270.001865850.001865850.001865850.0018...
2017-12-21 02:26:00                                 0.428521749.98000629
2017-12-21 02:27:00    0.517341400.020587980.001865850.005597570.0018...
2017-12-21 02:28:00    0.017000000.001865850.001865850.001865850.0055...
2017-12-21 02:29:00    0.001865850.001865850.001865850.001865850.0018...
2017-12-21 02:30:00    0.024329680.001865740.001865740.001865740.0018...
2017-12-21 02:31:00    0.001865740.001865740.001865740.001865742.0000...
2017-12-21 02:32:00    0.001865740.001865740.001865740.001865740.0018...
2017-12-21 02:33:00    0.001865690.001865690.001865690.001865690.0018...
2017-12-21 02:34:00    0.001865620.001865620.001865620.001865620.0018...
2017-12-21 02:35:00    0.001865640.00186564